<a href="https://colab.research.google.com/github/JibbyGeorge-DB/llm_journey/blob/main/Webscraper_Summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import requests
#from dotenv import load_dotenv
from IPython.display import Markdown, display
from bs4 import BeautifulSoup
from openai import OpenAI

In [13]:
#load_dotenv(override=True)
#api_key = os.getenv('OPENAI_API_KEY')

#for Google Colab
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them")
else:
    print("API key found and looks good!")

API key found and looks good!


In [ ]:
message = "tell me about Beautiful Soup package in python"
messages =[{"role": "user", "content": message }]

#Connecting to OpenAI
openai = OpenAI(api_key=api_key)
response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
response.choices[0].message.content

In [14]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [17]:
jg = Website("https://jibbygeorge.com.au")
print(jg.title)
#print(jg.text)

Jibby George - Portfolio


In [24]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."


In [19]:
# A function that writes a User Prompt that asks for summaries of websites:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(jg))

In [21]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [26]:
def summarize(url):
  website = Website(url)
  response = openai.chat.completions.create(
      model = "gpt-5-nano",
      messages = messages_for(website)
  )
  return response.choices[0].message.content


In [27]:
summarize("https://jibbygeorge.com.au")

'### Summary of Jibby George - Portfolio\n\nA personal portfolio site for Jibby George, a cloud consultant and solutions architect with 18+ years of experience in on-prem, hybrid cloud, and public cloud (Azure and AWS). The site showcases services, projects, blog posts, certifications, and contact options.\n\n- About\n  - Specialist in secure, scalable, and well-governed cloud solutions\n  - Focus areas: infrastructure automation, modern AI platform integration, and sustainable operational resilience\n\n- Work Highlights\n  - Cloud Migration: Enterprise-scale migration to Azure and AWS\n  - Azure Landing Zone: Secure, scalable enterprise Azure foundation\n  - Microsoft Foundry: AI-powered platform leveraging Microsoft Foundry technologies\n\n- Blog (Latest)\n  - AI Stack (Jan 7, 2026)\n  - Deploying Ollama Locally (Jan 7, 2026)\n\n- Certifications\n  - NVIDIA-Certified Associate: AI Infrastructure and Operations (2025)\n  - AWS Certified AI Practitioner (2025)\n  - Azure Administrator 